In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras import backend as K

import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
import time

In [2]:
x_train = []
#x_test = []
y_train = []

df_train = pd.read_csv('train.csv')
#df_test = pd.read_csv('../input/sample_submission.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values[:40479], miniters=1000):
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)

#for f, tags in tqdm(df_test.values, miniters=1000):
#    img = cv2.imread('../input/test-jpg/{}.jpg'.format(f))
#    x_test.append(cv2.resize(img, (32, 32)))
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32) / 255.
#x_test  = np.array(x_test, np.float32) / 255.

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [00:53<00:00, 755.21it/s]

(40479, 32, 32, 3)
(40479, 17)


In [3]:
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.2)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(32383, 32, 32, 3)
(32383, 17)
(8096, 32, 32, 3)
(8096, 17)


In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(48, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

In [21]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=[])
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]
model.fit(x = x_train, y= y_train, validation_data=(x_val, y_val),
                  batch_size=128,verbose=2, epochs=1,callbacks=callbacks,
                  shuffle=True)

y_pred = model.predict(x_val, batch_size = 128, verbose=2)
print("F2 score:",fbeta_score(y_val, np.array(y_pred)>0.2, beta=2, average='samples'))

Train on 32383 samples, validate on 8096 samples
Epoch 1/1
4s - loss: 0.1645 - val_loss: 0.1574
F2 score: 0.846135361192
